In [1]:
import function.functions as functions
# import function.functions_nocuda as functions # use this if you can't use cuda and the GPU

from qutip import *
import numpy as np
import sys

In [2]:
# Set device and parameters in this part

############################# Set parameters ##############################
device = 'charge_qubit' # change here to change the device: charge_qubit, flopping_spin, flopping_charge, transmon, fluxonium

data = np.load('data/params/'+device+'.npz', allow_pickle=True)
H_sys, drive_op, wq, g, kappa, num_A, dim = data['H_sys'], data['drive_op'], data['wq'], data['g'], data['kappa'], data['num_A'], data['dim']
#############################################################################

########################### Parameters of the paper ###########################
if device == 'charge_qubit':
    num_w = 151

    wlist = np.linspace(0,2,num_w,endpoint=True)
    Alist = [0.05*wq,0.5*wq,1.5*wq,2.45*wq]
    
    num_A = 300

    ground, excited = 0, 1

elif device == 'flopping_spin':
    num_w = 151

    wlist = np.linspace(0,2*wq,num_w,endpoint=True)
    Alist = [0.2*wq, 0.3*wq]
    
    N_rep =  10 # this means that we will have 2*N_rep+1 replicas

    ground, excited = 0, 1

elif device == 'transmon':
    num_w = 101

    wlist = np.linspace(0,2*wq,num_w,endpoint=True)
    Alist = [0.037*wq, 0.09*wq]

    N_rep =  25 # this means that we will have 2*N_rep+1 replicas
    n_states = 25

    Ejeff, EC = data['Ejeff'], data['EC']

    ground, excited = 0, 1

elif device == 'fluxonium':
    num_w = 101

    wlist = np.linspace(0,3*wq,num_w,endpoint=True)
    Alist = [0.6*wq, 1.2*wq]

    N_rep =  15 # this means that we will have 2*N_rep+1 replicas

    ground, excited = 0, 1

    n_states = 10

else:
    print('select a valid device')
    sys.exit()

qubit_state_list = [ground, excited]
################################################################################

########################### Custom parameters #################################
# test other parameters A_q, ground, excited, w_r, w_r_disp, compensation

save_file = True # test mode or save data to generate the data of the figure of the paper
################################################################################

In [3]:
def generate_g_parallel(H_sys, drive_op, n_states, N_rep, device):
    H_sys = Qobj(H_sys[0:n_states,0:n_states])
    drive_op = Qobj(drive_op[0:n_states,0:n_states])

    if save_file:
        fname = 'data/'+device+'/g_parallel/N_rep='+str(N_rep)+'_n_states='+str(n_states)\
                +'_dim='+str(dim)+'_num_w='+str(num_w)

        data = open(fname+'.txt', "w")
        data.close()

    else:
        fname = None

    g_parallel = np.zeros((len(wlist), len(Alist)), dtype=np.complex128)
    g_parallel_0 = np.zeros((len(wlist), len(Alist)))

    A_list, dd_real, dd2_real, z0_list = functions.get_derivatives(N_rep,Alist,H_sys,drive_op,wlist,n_states,num_A,fname,True,qubit_state_list)

    for i,A_q in enumerate(Alist):

        index_A = np.abs(A_list[0:(num_A-2)]-A_q).argmin()

        message = 'derivatives of the spectrum computed in A_q/w_q='+str(A_list[index_A]/wq)
        print(message)
        
        with open(fname+'.txt',"a") as data:
            data.write(message+"\n")
            
        g_parallel[:,i] = 1/2*g*(dd_real[:,index_A,1]-dd_real[:,index_A,0])

    for i,A_q in enumerate(Alist):

        if device == 'charge_qubit':
            g_parallel_0[:, i] = g*A_q*wq*np.ones(len(wlist))/((wq*np.ones(len(wlist)))**2-wlist**2)

        elif device == 'transmon':
            n01_modulo_squared = 1/2*np.sqrt(Ejeff/(8*EC))
            
            evals_analytical = np.zeros((len(wlist), n_states))
            evals_floquet_analytical = np.zeros((len(wlist), 2))

            for j in range(n_states):
                evals_analytical[:, j] = (-Ejeff + np.sqrt(8*Ejeff*EC)*(j+1/2) - EC/12*(6*j**2 + 6*j + 3))*np.ones(len(wlist))
                
            evals_floquet_analytical[:, 0] = evals_analytical[:,0] \
                - n01_modulo_squared*wq*np.ones(len(wlist))/(wq**2*np.ones(len(wlist)) - wlist**2)
            evals_floquet_analytical[:, 1] = evals_analytical[:,1] \
                + n01_modulo_squared*(wq*np.ones(len(wlist))/(wq**2*np.ones(len(wlist)) - wlist**2)*\
                    -2*(wq-EC)*np.ones(len(wlist))/((wq-EC)**2*np.ones(len(wlist))-wlist**2))

            g_parallel_0[:, i] = g*A_q/2*(n01_modulo_squared*(wq*np.ones(len(wlist))/(wq**2*np.ones(len(wlist)) - wlist**2)-2\
                *(wq-EC)*np.ones(len(wlist))/((wq-EC)**2*np.ones(len(wlist))-wlist**2))\
                    +n01_modulo_squared*wq*np.ones(len(wlist))/(wq**2*np.ones(len(wlist)) - wlist**2))

    if save_file:
        np.savez(fname, Alist=Alist, wlist=wlist, g=g, A_list=A_list, dd_real=dd_real, dd2_real=dd2_real,\
            g_parallel=g_parallel, g_parallel_0=g_parallel_0, z0_list=z0_list)

In [4]:
n_states_list = [2]
N_rep_list = [60] # this means that we will have 2*N_rep+1 replicas

for n_states in n_states_list:
    for N_rep in N_rep_list:
        generate_g_parallel(H_sys, drive_op, n_states, N_rep, device)

derivatives of the spectrum computed in A_q/w_q=0.2
derivatives of the spectrum computed in A_q/w_q=0.30000000000000004
derivatives of the spectrum computed in A_q/w_q=0.2
derivatives of the spectrum computed in A_q/w_q=0.30000000000000004
